In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip '/content/gdrive/MyDrive/enron11.zip' -d '/content'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
print("============check cpu config================")

!cat /proc/cpuinfo | grep model\ name
print("=============check ram config===============")

!cat /proc/meminfo | grep MemTotal

In [ ]:
#data processing


import random
import tensorflow as tf
seed = 1

import random
import numpy as np
import tensorflow as tf
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# importing system libraries

from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest
nltk.download('stopwords')
import time
start_time = time.time()



from nltk.test.classify_fixt import setup_module
setup_module()
from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
from sklearn.model_selection import cross_val_predict
# importing additional libraries

import pandas as pd
import sklearn as sk
import nltk
import pytest




from nltk.test.classify_fixt import setup_module
setup_module()

import pandas as pd
import numpy as np
import re
import collections

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle





# Read the whole data from the Enron Dataset into a variable allData.

pathwalk = walk(r"/content/enron1/")

allHamData, allSpamData = [], []
for root, dr, file in pathwalk:
    if 'ham' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allHamData.append(" ".join(ip.readlines()))
                
    elif 'spam' in str(file):
        for obj in file:
            with open(root + '/' + obj, encoding='latin1') as ip:
                allSpamData.append(" ".join(ip.readlines()))
                
                
                

allHamData = list(set(allHamData))
allSpamData = list(set(allSpamData))


hamPlusSpamData = allHamData + allSpamData
labels = ["ham"]*len(allHamData) + ["spam"]*len(allSpamData)

raw_df = pd.DataFrame({"email": hamPlusSpamData, 
                       "label": labels})



raw_df.sample(5)
raw_df['email']

""""""""""""""""""""""""""""""""""""""""""


def preprocess(data):
    # tokenization
    tokens = nltk.word_tokenize(data)
    tokens = [w.lower() for w in tokens if w.isalpha()]

    # finding uncommon words
    cnt = Counter(tokens)
    uncommons = cnt.most_common()[:-int(len(cnt)*0.1):-1]
    
    # listing stopwords from NLTK
    stops = set(nltk.corpus.stopwords.words('english'))

    # removing stop words and uncommon words
    tokens = [w for w in tokens if (w not in stops and w not in uncommons)]

    # lemmatization
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w, pos='a') for w in tokens]

    return tokens

label_encoder = sk.preprocessing.LabelEncoder()
raw_df['label'] = label_encoder.fit_transform(raw_df.label)
import string
from nltk.corpus import stopwords

# deleting stopwords like a the this that
def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    return ' '.join([word for word in no_punc.split() if word.lower() not in stopwords.words('english')])

raw_df['email']=raw_df['email'].apply(process_text)
raw_df['email']

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])
raw_df['email']=raw_df['email'].apply(stemming)
raw_df.head()


from sklearn.feature_extraction.text import CountVectorizer
vectorizer= CountVectorizer()
message_bow = vectorizer.fit_transform(raw_df['email'])

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow,raw_df['label'],test_size=0.25)

X = message_bow
y = raw_df['label']




# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(message_bow)

# Transform entire BoW into tf-idf corpus
message_bow_tfidf = tfidf_transformer.transform(message_bow)

X_tfidf = message_bow_tfidf
y = raw_df['label']
X_tfidf_train,X_tfidf_test,y_tfidf_train,y_tfidf_test = train_test_split(message_bow_tfidf,raw_df['label'],test_size=0.25)



texts1 = np.asarray(message_bow)
labels1 = np.asarray(raw_df['label'])





In [ ]:

#Naive Bayesian
#Support Vector Machine
#Random Forest



print("this is naive_bayes part")
print("=======================================================================================================")
from sklearn.naive_bayes import MultinomialNB
nb= MultinomialNB()
nb.fit(X_train,y_train)
y_pred = (nb.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import classification_report



from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.model_selection import cross_val_score


nb3= MultinomialNB()
accuracies_nb_accuracy = cross_val_score(estimator=nb3, scoring="accuracy", 
    X=X, y=y, cv=5)
accuracies_nb_f1 = cross_val_score(estimator=nb3, scoring="f1", 
    X=X, y=y, cv=5)
accuracies_nb_recall = cross_val_score(estimator=nb3, scoring="recall", 
    X=X, y=y, cv=5)
accuracies_nb_precision = cross_val_score(estimator=nb3, scoring="precision", 
    X=X, y=y, cv=5)

print("NB corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_nb_accuracy.mean(), accuracies_nb_accuracy.std()))
print("NB corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_nb_f1.mean(), accuracies_nb_f1.std()))
print("NB corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_nb_recall.mean(), accuracies_nb_recall.std()))
print("NB corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_nb_precision.mean(), accuracies_nb_precision.std()))



print("this is svm part")
print("=======================================================================================================")

from sklearn import svm

from sklearn.metrics import confusion_matrix

from sklearn.model_selection import cross_val_score


svmc3= svm.SVC()
accuracies_svmc_accuracy = cross_val_score(estimator=svmc3, scoring="accuracy", 
    X=X_tfidf, y=y, cv=5)
accuracies_svmc_f1 = cross_val_score(estimator=svmc3, scoring="f1", 
    X=X_tfidf, y=y, cv=5)
accuracies_svmc_recall = cross_val_score(estimator=svmc3, scoring="recall", 
    X=X_tfidf, y=y, cv=5)
accuracies_svmc_precision = cross_val_score(estimator=svmc3, scoring="precision", 
    X=X_tfidf, y=y, cv=5)

print("SVM corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_svmc_accuracy.mean(), accuracies_svmc_accuracy.std()))
print("SVM corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_svmc_f1.mean(), accuracies_svmc_f1.std()))
print("SVM corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_svmc_recall.mean(), accuracies_svmc_recall.std()))
print("SVM corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_svmc_precision.mean(), accuracies_svmc_precision.std()))




print("this is Random Forest part")
print("=======================================================================================================")
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


R_tree = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
R_tree.fit(X_tfidf_train, y_tfidf_train)



y_pred = R_tree.predict(X_tfidf_test)
from sklearn.metrics import classification_report



from sklearn.metrics import confusion_matrix


from sklearn.model_selection import cross_val_score


R_tree3 = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, max_depth=100, random_state = 42)
accuracies_rtree_accuracy = cross_val_score(estimator=R_tree3, scoring="accuracy", 
    X=X_tfidf, y=y, cv=10)
accuracies_rtree_f1 = cross_val_score(estimator=R_tree3, scoring="f1", 
    X=X_tfidf, y=y, cv=10)
accuracies_rtree_recall = cross_val_score(estimator=R_tree3, scoring="recall", 
    X=X_tfidf, y=y, cv=10)
accuracies_rtree_precision = cross_val_score(estimator=R_tree3, scoring="precision", 
    X=X_tfidf, y=y, cv=10)

print("RF corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_rtree_accuracy.mean(), accuracies_rtree_accuracy.std()))
print("RF corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_rtree_f1.mean(), accuracies_rtree_f1.std()))
print("RF corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_rtree_recall.mean(), accuracies_rtree_recall.std()))
print("RF corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_rtree_precision.mean(), accuracies_rtree_precision.std()))




In [ ]:


print("this is artifacial neural network part")
print("=======================================================================================================")

# Part 2 - Now let's make the ANN!
X = message_bow_tfidf
y = raw_df['label']




from tensorflow.keras.optimizers import SGD




from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)


X_train_sc = sc.fit(X_train)
X_train = X_train_sc.transform(X_train)
X_test_sc = sc.fit(X_test)
X_test = X_test_sc.transform(X_test)

X_train_ann = X_train.copy()
X_train_ann = X_train_ann.toarray()




X_ann = X.copy()
X_ann = X_ann.toarray()

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 50358))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
annn = classifier.compile(optimizer = 'RMSprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set

ann = classifier.fit(X_train_ann, y_train, batch_size = 16, epochs = 1, shuffle=False)


# Predicting the Test set results
y_pred = (classifier.predict(X_test) > 0.5).astype("int32")
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix



from sklearn.metrics import classification_report
print("ann score",classification_report(y_test,y_pred, digits=4))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


from tensorflow.keras.optimizers import RMSprop


def create_keras_model():

    model = Sequential()

    model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 50358))
    model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    rmsprop = RMSprop()
  #  rmsprop = RMSprop(learning_rate=0.001)
    # Compiling the ANN
    model.compile(optimizer = rmsprop, loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
    # Fitting the ANN to the Training set



    # Predicting the Test set results

    return model


from keras.wrappers.scikit_learn import KerasClassifier

ann2 = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=16, epochs=15)


ann2._estimator_type = "classifier" 


accuracies_ann_accuracy = cross_val_score(estimator=ann2, scoring="accuracy", 
    X=X_ann, y=y, cv=10)
accuracies_ann_f1 = cross_val_score(estimator=ann2, scoring="f1", 
    X=X_ann, y=y, cv=10)
accuracies_ann_recall = cross_val_score(estimator=ann2, scoring="recall", 
    X=X_ann, y=y, cv=10)
accuracies_ann_precision = cross_val_score(estimator=ann2, scoring="precision", 
    X=X_ann, y=y, cv=10)

print("ann corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_ann_accuracy.mean(), accuracies_ann_accuracy.std()))
print("ann corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_ann_f1.mean(), accuracies_ann_f1.std()))
print("ann corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_ann_recall.mean(), accuracies_ann_recall.std()))
print("ann corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_ann_precision.mean(), accuracies_ann_precision.std()))




In [ ]:
#LSTM

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import keras
from keras.models import Sequential
from keras.layers import Dense


from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# number of words used as features
max_features = 500000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500


# we will use 80% of data as training, 20% as validation data
training_samples = int(4994 * .75)
validation_samples = int(4994 - training_samples)
# sanity check
print(len(raw_df['email']) == (training_samples + validation_samples))
print("The number of training {0}, validation {1} ".format(training_samples, validation_samples))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_df['email'])
sequences = tokenizer.texts_to_sequences(raw_df['email'])

word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

print("data shape: ", data.shape)



# shuffle data
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = np.asarray(labels)
labels = y[indices]


texts_train = data[:training_samples]
y_train = labels[:training_samples]
texts_test = data[training_samples:]
y_test = labels[training_samples:]


from tensorflow.keras.optimizers import RMSprop


max_features =500000
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])
history_ltsm = model.fit(texts_train, y_train, epochs=1, batch_size=16, shuffle=False)

pred = (model.predict(texts_test) > 0.5).astype("int32")

acc = model.evaluate(texts_test, y_test)
proba_ltsm = (model.predict(texts_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
print("Test loss is {0:.4f} accuracy is {1:.4f}  ".format(acc[0],acc[1]))

from sklearn.metrics import classification_report
print("LSTM score",classification_report(y_test,pred, digits=4))
print(confusion_matrix(pred, y_test))

from pandas.core.common import random_state

from keras.layers.core import Flatten



def create_keras_model():

    max_features =500000
    model2 = Sequential()
    model2.add(Embedding(max_features, 128))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128, return_sequences=True))
    model2.add(LSTM(128))
    model2.add(Dense(128, activation='relu'))
    model2.add(Dense(1, activation='sigmoid'))
#    rmsprop1 = RMSprop(learning_rate = 0.1)
    model2.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])



    return model2


from keras.wrappers.scikit_learn import KerasClassifier

LSTMmodel1 = KerasClassifier(
        build_fn=create_keras_model, 
        batch_size=16, epochs=10)


LSTMmodel1._estimator_type = "classifier"






from sklearn.model_selection import cross_val_score


accuracies_LSTMmodel1_accuracy = cross_val_score(estimator=LSTMmodel1, scoring="accuracy", 
    X=data, y=labels, cv=10)
accuracies_LSTMmodel1_f1 = cross_val_score(estimator=LSTMmodel1, scoring="f1", 
    X=data, y=labels, cv=10)
accuracies_LSTMmodel1_recall = cross_val_score(estimator=LSTMmodel1, scoring="recall", 
    X=data, y=labels, cv=10)
accuracies_LSTMmodel1_precision = cross_val_score(estimator=LSTMmodel1, scoring="precision", 
    X=data, y=labels, cv=10)

print("LSTM")
print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("LSTMmodel1 corss validation :  %0.4f accuracy  with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_accuracy.mean(), accuracies_LSTMmodel1_accuracy.std()))
print("LSTMmodel1 corss validation :  %0.4f f1     with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_f1.mean(), accuracies_LSTMmodel1_f1.std()))
print("LSTMmodel1 corss validation :  %0.4f recall   with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_recall.mean(), accuracies_LSTMmodel1_recall.std()))
print("LSTMmodel1 corss validation :  %0.4f precision with a standard deviation of %0.4f" % (accuracies_LSTMmodel1_precision.mean(), accuracies_LSTMmodel1_precision.std()))


